# 패키지 설치

In [ ]:
# 필요한 패키지 설치
!pip install hanja   # 한자를 한글로 변환하기 위한 패키지
!pip install nltk   # 불용어 제거를 위한 패키지
!pip install git+https://github.com/haven-jeon/PyKoSpacing.git   # 띄어쓰기 바로 잡기 위한 패키지
!pip install git+https://github.com/ssut/py-hanspell.git   # 오타 및 맞춤법을 수정해주는 패키지

# 라이브러리 호출

In [ ]:
# 필요한 라이브러리 호출
import pandas as pd
import re
import hanja
from hanspell import spell_checker
from nltk.corpus import stopwords
from pykospacing import Spacing

# 불용어 목록 파일 생성

In [ ]:
# NLTK 불용어 다운로드
import nltk
nltk.download('stopwords')
# stopwords 폴더 안에 korean 불용어 파일 만들고 노션에 있는거 복붙해서 넣기

# 코랩 사용할 경우 파일 옮기는 방법

In [ ]:
# 주피터가 아니라 코랩 쓰는 경우에만 참고
#지연) 구글 코랩으로 작성해서 직접 생성한 korean파일을 옮기는 코드가 필요했음.. 주피터로 하면 필요없을듯!
# import shutil

# # 파일 경로
# src_path = '/content/korean'
# dest_path = '/root/nltk_data/corpora/stopwords'

# # 파일 이동
# shutil.move(src_path, dest_path)

### 정규표현식 테스트 ..

In [ ]:
# # 정규표현식 테스트
# patterns = [
#     r'<.*?>|\[.*?\]|【.*?】|\{.*?\}',
#     r'[^가-힣0-9\s◆◇▲▼]'
# ]

# text = '안녕하세요! [안녕하세요] (안녕하세요★☆)'
# for pattern in patterns:
#     text = re.sub(pattern, '', text)
# print(text)

In [ ]:
# text1 = '안녕하세요.이것은 테스트 문장입니다...그렇습니다.'
# text2 = '안녕하세요.. 이것은 테스트 문장입니다. 그렇습니다.'

# def split_text(article):   # split하는 함수 정의
#     sentences = re.split(r'(?<!\.)\.(?!\.)', article)  # 마침표를 기준으로 문장 나누기, 단 마침표가 2개 이상 연달아 있는 경우 제외
#     sentences = [sentence.strip() for sentence in sentences if sentence.strip()]   # 공백 제거 및 빈 문장 필터링
#     return sentences

# print(split_text(text1))
# print('=====')
# print(split_text(text2))

# 전처리 함수 정의

In [ ]:
# 데이터 클렌징 함수 정의
def clean_text(text):

    # 0. 한자를 한글로 변환
    # text = re.sub(r'\([^()]*\)', '', text)  # 괄호 안의 한자는 삭제
    text = hanja.translate(text, 'substitution')  # 그 외 한자는 한글로 변환

    # 1. 첫 번째 문장에서 `◆...◆` 패턴 제거
    sentences = text.split('. ')  # 마침표와 공백을 기준으로 문장을 나누기
    if sentences:  # 문장이 존재할 경우
        sentences[0] = re.sub(r'◆.*?◆', '', sentences[0])  # 첫 번째 문장에서 패턴 제거
    text = '. '.join(sentences)  # 문장들을 다시 결합

    # 2. 제거할 단어 및 패턴 목록
    # 기사 전체 제거, 패턴 제거 순으로 함수 실행하기
    patterns = [
        r'[^.]*※[^.]*\.',            # '※'를 포함하는 문장 전체를 제거
        r'[^.]*\[사설\][^.]*\.',      # '[사설]'을 포함하는 문장 전체를 제거
        r'[^.]*마켓PRO[^.]*\.',       # '마켓PRO'를 포함하는 문장 전체를 제거
        r'[^.]*한경유레카[^.]*\.',   # '한경유레카'를 포함하는 문장 전체를 제거
        r'[^.]*이 기사는[^.]*\.',    # '이 기사는'을 포함하는 문장 전체를 제거
        r'[^.]*@[^.]*\.',            # '@'를 포함하는 문장 전체를 제거 / 이메일을 제거하기 위함
        r'[^.]*#장면[^.]*\.',          # '#장면'을 포함하는 문장 전체를 제거
        r'[^.]*\[기자\][^.]*\.',      # '[기자]'를 포함하는 문장 전체를 제거
        r'[^.]*\[저작권\][^.]*\.',    # '[저작권]'을 포함하는 문장 전체를 제거
        r'[^.]*\[퀴즈\][^.]*\.',       # '[퀴즈]'를 포함하는 문장 전체를 제거
        r'[^.]*<용 어>[^.]*\.',
        r'지금은 임원시대',            # '지금은 임원시대' 라는 문자열만 제거
        r'<.*?>|\[.*?\]|【.*?】|\{.*?\}', # ()제외, 온갖 종류의 괄호와 그 안의 내용 전부 제거
        r'레이더M 기사 더보기>>>',
        r'미래를 바꿀 혁신기술 12가지 - ',
        r'MBA도 모바일로 공부한다.',
        r'매경이 전하는 세상의 지식\(매-세-지',
        r'▶ 네이버 뉴스스탠드에서 매일경제를 MY뉴스로 구독하세요',
        r'▶뉴스 이상의 무궁무진한 프리미엄 읽을거리',
        r'▶아나운서가 직접 읽어주는 오늘의 주요 뉴스',
        r'▶매경 뉴스레터 \'매콤달콤\'을 지금 구독하세요'
        r'▶\'지킬앤하이드\' 조승우 공연 티켓 경품 이벤트',
        r'▶네이버 메인에서 \'매일경제\'를 받아보세요',
        r'▶뉴스레터 \'매콤달콤\' 구독',
        r'▶무궁무진한 프리미엄 읽을거리',
        r'▶네이버에서 \'매일경제\' 뉴스 구독하고 경품 받아가세요',
        r'▶매경이 에어팟프로 쏩니다! \'M코인\'',
        r'▶기사공유하고 코인적립하세요 \'M코인\'',
        r'▶\'M코인\' 지금 가입하면 5000코인 드려요',
        r'▶\'매일경제\' 바로가기',
        r'▶매부리TV 구독하고 에어팟프로 득템!!',
        r'▶제21회 세계지식포럼 : 팬데노믹스',
        r'▶ 팬데믹위기 해법 찾는다! - 제21회 세계지식포럼',
        r'▶ 궁금한 제조과정 영상으로 보세요. \'이렇게 만들죠\'',
        r'▶ 데일리 뉴스 브리핑 \'매경이 전하는 세상의 지식\'',
        r'▶ 매일매일 색다른 뉴스레터 \'매콤달콤\' 구독하세요',
        r'▶ 아파트 살까 청약할까. 여기서 확인하세요. \'매부리tv\'',
        r'▶ \'경제 1위\' 매일경제, 네이버에서 구독하세요'
        r'▶ 이 제품은 \'이렇게 만들죠\' 영상으로 만나요',
        r'▶ 부동산의 모든것 \'매부리TV\'가 펼칩니다',
        r'▶ \'경제 1위\' 매일경제, 앱으로 편하게 보세요',
        r'▶ 매일경제 지식레터 \'매콤달콤\' 받아보세요',
        r'▶ 매경이 알려주는 \'취업비법\' 한달간 무료'
    ]

    # 단어 및 패턴을 제거하는 함수
    def remove_patterns(text):
        for pattern in patterns:
            text = re.sub(pattern, '', text)
        return text

    # 패턴 제거
    # text = remove_article(text)
    text = remove_patterns(text)

    # 3. 문장 단위로 split하여 리스트에 저장
    def split_text(article):   # split하는 함수 정의
        sentences = re.split(r'(?<!\.)\.(?!\.)', article)  # 마침표를 기준으로 문장 나누기, 단 마침표가 2개 이상 연달아 있는 경우 제외
        sentences = [sentence.strip() for sentence in sentences if sentence.strip()]   # 공백 제거 및 빈 문장 필터링
        return sentences

    # 각 문장들을 요소로 갖는 리스트인 text_list
    text_list = split_text(text)

    # 4. 특수 문자 제거 (◆, ◇, ▲, ▼ 제외)
    text_list = [re.sub(r'[^가-힣0-9\s◆◇▲▼~,.\'"%]', '', s) for s in text_list]  # 한글과 숫자, 공백, 그리고 ◆, ◇, ▲, ▼, ~, ', ", . , , % 만 남김

    # 5. 띄어쓰기 교정
    spacing = Spacing()
    text_list = [spacing(s) for s in text_list]

    # # 6. 오타 수정
    # def correct_text(sentence):
    #     result = spell_checker.check(sentence)
    #     return result.checked   # 수정된 문장 반환
    # # 직접 spell_checker 라이브러리 코드 열어서 payload의 key값에 passportKey와 _callback 추가, value는 네이버 맞춤법 검사기 개발자 도구 참고하기
    # # data 변수도 바꿔줘야함 -> 자세한건 노션에 서술

    # text_list = [correct_text(s) for s in text_list]

    #아래 오류가 났을 때의 코드 -> API 응답이 뜨지 않고 오류 발생이라고 뜨면 라이브러리 코드를 수정 안했거나 잘못한 것
    '''
    def correct_typos(sentence):
        if not isinstance(sentence, str):
            print("입력값이 문자열이 아닙니다.")
            return sentence  # 문자열이 아닐 경우 원본 반환
        try:
            result = spell_checker.check(sentence)
            if not result.result:
                print(f"수정 실패: {result.original}")
                return sentence  # 수정 실패 시 원본 문장 반환
            return result.checked  # 수정된 문장 반환
        except Exception as e:
            print(f"오류 발생: {e}")
            return sentence  # 오류 발생 시 원본 문장 반환

    corrected_text_list = []
    for sentence in text_list:
        try:
            result = spell_checker.check(sentence)
            print(f"API 응답: {result}")  # API 응답 확인
            corrected_text_list.append(result.checked)
        except Exception as e:
            print(f"오류 발생: {e}")
            corrected_text_list.append(sentence)  # 오류 발생 시 원본 문장 반환

    for original, corrected in zip(text_list, corrected_text_list):
        print(f"원본: {original}\n수정: {corrected}\n")
    '''

    # 7. 불용어 제거
    stop_words = set(stopwords.words('korean'))   # korean 불용어 파일 만들어져 있어야 함
    def remove_stopwords(text):   # 불용어 제거 함수 정의
        return ' '.join([word for word in text.split() if word not in stop_words])
    text_list = [remove_stopwords(s) + '.' for s in text_list] # 문장 마지막에 다시 온점 붙여서 반환

    # 8. 분리된 문장들 다시 하나의 텍스트로 합치기
    text = ' '.join(text_list)   # 다시 하나의 문자열로 합쳐진걸 원하면 return text 하면 됨

    return text

# 특정 연도 csv 파일 하나만 돌릴 때 아래 코드 사용

In [ ]:
import time

# CSV 파일을 데이터프레임으로 불러오기
start = time.time()
df = pd.read_csv('2005_매일경제.csv')

# 데이터 정제 ) null값과 중복 제거
df.dropna(subset=['body'], inplace=True)
df.drop_duplicates(inplace=True)

# 패턴 목록 정의
patterns = [
    r'삼성서울병원',
    r'삼성 서울병원',
    r'삼성안내견센터',
    r'삼성홀',
    r'삼성전자홀',
    r'스타크래프트',
    r'프로리그',
    r'라이온즈',
    r'매경TEST',
    r'<표>',
    r'\[표\]',
    r'\[포토\]',
    r'\[MK 추천매물\]'
]

# 패턴이 포함된 행을 삭제하기 위한 조건식 작성
condition1 = df['title'].apply(lambda text: any(re.search(pattern, text) for pattern in patterns))
df = df[~condition1]
condition2 = df['body'].apply(lambda text: any(re.search(pattern, text) for pattern in patterns))
df = df[~condition2]

# 'title'과 'body' 컬럼에 클렌징 함수 적용
df['cleaned_title'] = df['title'].apply(clean_text)
df['cleaned_body'] = df['body'].apply(clean_text)

# 결과 확인
print(df[['body', 'cleaned_body']].head())

# 클렌징된 데이터프레임 저장 (원하는 파일명으로 변경)
df.to_csv('cleaned_2005_매일경제.csv', index=False, encoding='utf-8-sig')

end = time.time()
print(end - start)

# 여러 연도 csv 파일 반복문으로 돌릴 때 아래 코드 사용

In [ ]:
# 파일 경로 및 범위 설정
import os

start_year = 2005
end_year = 2024

# 모든 연도의 CSV 파일을 처리
for year in range(start_year, end_year + 1):
    filename = f'/{year}_매일경제.csv'

    if os.path.exists(filename):
        print(f"Processing file: {filename}")

        # CSV 파일을 데이터프레임으로 불러오기
        df = pd.read_csv(filename)

        df.dropna(subset=['body'], inplace=True)
        df.drop_duplicates(inplace=True)

        # 패턴 목록 정의
        patterns = [
            r'삼성서울병원',
            r'삼성 서울병원',
            r'삼성안내견센터',
            r'삼성홀',
            r'삼성전자홀',
            r'스타크래프트',
            r'프로리그',
            r'라이온즈',
            r'매경TEST',
            r'<표>',
            r'\[표\]',
            r'\[포토\]',
            r'\[MK 추천매물\]'
        ]

        # 패턴이 포함된 행을 삭제하기 위한 조건식 작성
        condition1 = df['title'].apply(lambda text: any(re.search(pattern, text) for pattern in patterns))
        df = df[~condition1]
        condition2 = df['body'].apply(lambda text: any(re.search(pattern, text) for pattern in patterns))
        df = df[~condition2]

        # 'body'와 'title' 컬럼에 클렌징 함수 적용
        df['cleaned_title'] = df['title'].apply(clean_text)
        df['cleaned_body'] = df['body'].apply(clean_text)

        # 클렌징된 데이터프레임 저장
        output_filename = f'cleaned_{year}_매일경제.csv'  # 파일명이 한국경제로 되어있길래 수정함
        df.to_csv(output_filename, index=False, encoding='utf-8-sig')

        print(f"Saved cleaned file: {output_filename}")

    else:
        print(f"File not found: {filename}")